In [10]:
'''
['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌(+/-)', '漲跌價差', 
'最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比']
['0050', '元大台灣50', '6,307,525', '5,434', '778,088,986', '122.20', '124.50', '122.00', '124.35', '<p style= color:red>+</p>', '2.10', 
'124.35', '18', '124.40', '142', '0.00']
''' 
import os
import json
from tqdm.notebook import tqdm, tnrange, tqdm_notebook
from clickhouse_driver import Client
from datetime import datetime
from datetime import timedelta, date

def insert_table(conn, insert_query, records):
    try:
        return conn.execute(insert_query, records)
    except Exception as err:
        print(err)
    return None

path = '2021'
pathTemplate = "{}/{}"
sessions = ["data9","data8"]
insert_query = "INSERT INTO twse.squote (skey, sdate, scode, sname, open, high, low, close, volume, transaction, svalue, diff) VALUES"

conn = Client(host='localhost',
              database='twse',
              user='default',
              password='',
              compression='lz4',
              secure=False
              )

files = os.listdir(path)
filesSorted = sorted(files)
for f_idx in tnrange(len(filesSorted), desc='Import Progress'):
    filename = filesSorted[f_idx]
    f_name, f_ext = os.path.splitext(filename)
    f = open(os.path.join(path, filename), encoding="utf8")
    data = json.load(f)
    if "OK" not in data["stat"]:
        continue
    records = []
    currentDate = datetime.strptime(f_name, "%Y%m%d").date()
    currentDateStr = currentDate.strftime("%Y-%m-%d")
    quotes = None
    for s in sessions:
        if s in data:
            quotes = data[s]
            break

    for r_idx in tnrange(len(quotes), desc=currentDateStr):
        row = quotes[r_idx]
        sign = 1
        if "green" in row[9]:
            sign = -1
        skey = f_name + row[0]
        my_dict = (skey,           # primary key
                    currentDate, # date 
                    row[0],         # code
                    row[1],         # name
                    float(row[5].replace("--","0").replace(",","")), # open
                    float(row[6].replace("--","0").replace(",","")), # high
                    float(row[7].replace("--","0").replace(",","")), # low
                    float(row[8].replace("--","0").replace(",","")), # close
                    int(row[2].replace(",","")), # volume
                    int(row[3].replace(",","")), # transaction
                    int(row[4].replace(",","")), # value
                    float(row[10].replace("--","0")) * sign # diff
                   )

        records.append(my_dict)
    cnt = insert_table(conn, insert_query, records)
    #print(currentDateStr, cnt, "Record inserted successfully.")
    f.close()
print('Completed.')

Import Progress:   0%|          | 0/1 [00:00<?, ?it/s]

2021-12-30:   0%|          | 0/25946 [00:00<?, ?it/s]

Completed.
